<a href="https://colab.research.google.com/github/MadhuShree-A/TIC-based-Image-Compression/blob/main/TIC_streamlit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip uninstall -y compressai
!pip install compressai==1.2.4
!pip install -q torch torchvision streamlit pyngrok pillow scikit-image nbformat

!git --version

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 kB 3.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for compressai: filename=compressai-1.2.4-cp312-cp312-linux_x86_64.whl size=296694 sha256=6ec9df5b0bb0da9c27a955af47c01447e9d25ff971dbd9d8c8f0da653adbd9df
  Stored in directory: /root/.cache/pip/wheels/74/3d/0b/34c8476f346efaa9328f44a850c8e4a59ba376c7690b743346
Successfully built compressai
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 83.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 121.8 MB/s eta 0:00:00
git version 2.34.1


In [2]:
import os, subprocess

repos = [
    'https://github.com/lumingzzz/TIC.git',      # community implementation (DCC 2022)
    'https://github.com/NJUVISION/TIC.git',     # possible author/org path
    'https://github.com/NJUVISION/MPA.git'      # general fallback
]

cloned = []
for r in repos:
    name = r.split('/')[-1].replace('.git','')
    target = f'/content/{name}'
    if os.path.exists(target):
        print('Already present:', target)
        cloned.append(target)
        continue
    try:
        print('Cloning', r)
        subprocess.run(['git','clone',r,target], check=True)
        print('Cloned to', target)
        cloned.append(target)
    except Exception as e:
        print(' Failed to clone', r, '->', e)

print('\nCloned repos:', cloned)


Cloning https://github.com/lumingzzz/TIC.git
Cloned to /content/TIC
Already present: /content/TIC
Cloning https://github.com/NJUVISION/MPA.git
Cloned to /content/MPA

Cloned repos: ['/content/TIC', '/content/TIC', '/content/MPA']


In [3]:
import glob

found_weights = []
for root, _, files in os.walk('/content'):
    for f in files:
        if f.endswith('.pth') or f.endswith('.pt'):
            found_weights.append(os.path.join(root, f))

if found_weights:
    print("✅ Found pretrained weight files:")
    for f in found_weights:
        print("  ", f)
else:
    print("⚠️ No pretrained TIC weights found — fallback to CompressAI pretrained model will be used.")


⚠️ No pretrained TIC weights found — fallback to CompressAI pretrained model will be used.


In [4]:
from pyngrok import ngrok
import getpass

print("👉 Visit https://dashboard.ngrok.com/get-started/your-authtoken to get your auth token.")
NGROK_AUTH_TOKEN = getpass.getpass("Paste your ngrok authtoken here: ")

ngrok.kill()
ngrok.set_auth_token(NGROK_AUTH_TOKEN)
print("✅ ngrok authtoken configured successfully.")

👉 Visit https://dashboard.ngrok.com/get-started/your-authtoken to get your auth token.
Paste your ngrok authtoken here: ··········
✅ ngrok authtoken configured successfully.


In [5]:
'''utils_py = r"""
import io, torch
import torchvision.transforms as T
from PIL import Image
import numpy as np
import torch.nn.functional as F
from skimage.metrics import peak_signal_noise_ratio as compare_psnr, structural_similarity as compare_ssim

def load_image_as_tensor(pil_img):
    transform = T.Compose([T.ToTensor()])
    return transform(pil_img).unsqueeze(0)

def tensor_to_pil(x):
    x = x.detach().cpu().clamp(0,1)
    if x.ndim == 4:
        x = x[0]
    arr = (x.numpy()*255).round().astype('uint8').transpose(1,2,0)
    return Image.fromarray(arr)

def compute_metrics(img1, img2):
    a, b = np.array(img1).astype('float32'), np.array(img2).astype('float32')
    psnr = compare_psnr(a, b, data_range=255)
    ssim = compare_ssim(a, b, data_range=255, channel_axis=-1)
    return psnr, ssim

def pad_to_multiple_of(x, div=64):
    _, _, h, w = x.shape
    new_h = int(np.ceil(h/div)*div)
    new_w = int(np.ceil(w/div)*div)
    return F.pad(x, (0, new_w-w, 0, new_h-h)), (h, w)

def load_model(device='cpu'):
    # ✅ Fallback: use pretrained CompressAI Cheng2020 model
    from compressai.zoo import cheng2020_anchor
    model = cheng2020_anchor(quality=6, pretrained=True).eval().to(device)
    return ('compressai_cheng2020', model)

def compress_and_decompress(model_info, pil_img, device='cpu'):
    tag, model = model_info
    x = load_image_as_tensor(pil_img).to(device)
    x_padded, orig_size = pad_to_multiple_of(x, div=64)
    with torch.no_grad():
        out_enc = model.compress(x_padded)
        comp_bytes = sum(len(s) for s in out_enc['strings'])
        out_dec = model.decompress(out_enc['strings'], out_enc['shape'])
        x_hat = out_dec['x_hat'].clamp(0,1)
    h, w = orig_size
    x_hat = x_hat[..., :h, :w]
    return x_hat, comp_bytes
"""
open("/content/utils.py","w").write(utils_py)
print(" Created utils.py")

# ============================================
# 🧩 STEP 6 — CREATE STREAMLIT APP FILE
# ============================================
app_py = r"""
import streamlit as st
import torch
from PIL import Image
import io
from utils import load_model, compress_and_decompress, tensor_to_pil, compute_metrics

st.set_page_config(page_title="Transformer Image Compression", layout="wide")
st.title("🧠 Deep Learning Image Compression Demo")
st.caption("Using pretrained CompressAI (Cheng2020) model")

uploaded = st.file_uploader("Upload an image", type=['png','jpg','jpeg'])

if uploaded:
    pil_img = Image.open(uploaded).convert('RGB')

    # 🔒 Limit image size to prevent OOM crash
    max_size = 512
    if pil_img.width > max_size or pil_img.height > max_size:
        pil_img.thumbnail((max_size, max_size))
        st.warning(f"Image resized to {pil_img.size} to prevent out-of-memory issues.")

    st.image(pil_img, caption="Original Image", use_column_width=True)

    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    if 'model_info' not in st.session_state:
        with st.spinner('Loading pretrained model...'):
            st.session_state['model_info'] = load_model(device)

    model_info = st.session_state['model_info']

    try:
        with st.spinner('Compressing and reconstructing...'):
            x_hat_tensor, comp_bytes = compress_and_decompress(model_info, pil_img, device)
            rec_img = tensor_to_pil(x_hat_tensor)
            psnr, ssim = compute_metrics(pil_img, rec_img)
            bpp = (comp_bytes * 8) / (pil_img.width * pil_img.height)

        col1, col2 = st.columns(2)
        col1.image(pil_img, caption="Original")
        col2.image(rec_img, caption="Reconstructed")

        st.markdown(f"**Bits per pixel:** {bpp:.4f}")
        st.markdown(f"**PSNR:** {psnr:.2f} dB")
        st.markdown(f"**SSIM:** {ssim:.4f}")

    except RuntimeError as e:
        st.error(f"Compression failed: {str(e)}. Try a smaller image.")
"""
open("/content/app.py","w").write(app_py)
print("✅ Created app.py")'''


'utils_py = r"""\nimport io, torch\nimport torchvision.transforms as T\nfrom PIL import Image\nimport numpy as np\nimport torch.nn.functional as F\nfrom skimage.metrics import peak_signal_noise_ratio as compare_psnr, structural_similarity as compare_ssim\n\ndef load_image_as_tensor(pil_img):\n    transform = T.Compose([T.ToTensor()])\n    return transform(pil_img).unsqueeze(0)\n\ndef tensor_to_pil(x):\n    x = x.detach().cpu().clamp(0,1)\n    if x.ndim == 4:\n        x = x[0]\n    arr = (x.numpy()*255).round().astype(\'uint8\').transpose(1,2,0)\n    return Image.fromarray(arr)\n\ndef compute_metrics(img1, img2):\n    a, b = np.array(img1).astype(\'float32\'), np.array(img2).astype(\'float32\')\n    psnr = compare_psnr(a, b, data_range=255)\n    ssim = compare_ssim(a, b, data_range=255, channel_axis=-1)\n    return psnr, ssim\n\ndef pad_to_multiple_of(x, div=64):\n    _, _, h, w = x.shape\n    new_h = int(np.ceil(h/div)*div)\n    new_w = int(np.ceil(w/div)*div)\n    return F.pad(x, (0, 

In [6]:
utils_py = r"""
import io, torch
import torchvision.transforms as T
from PIL import Image
import numpy as np
import torch.nn.functional as F
from skimage.metrics import peak_signal_noise_ratio as compare_psnr, structural_similarity as compare_ssim

def load_image_as_tensor(pil_img):
    transform = T.Compose([T.ToTensor()])
    return transform(pil_img).unsqueeze(0)

def tensor_to_pil(x):
    x = x.detach().cpu().clamp(0,1)
    if x.ndim == 4:
        x = x[0]
    arr = (x.numpy()*255).round().astype('uint8').transpose(1,2,0)
    return Image.fromarray(arr)

def compute_metrics(img1, img2):
    a, b = np.array(img1).astype('float32'), np.array(img2).astype('float32')
    psnr = compare_psnr(a, b, data_range=255)
    ssim = compare_ssim(a, b, data_range=255, channel_axis=-1)
    return psnr, ssim

def pad_to_multiple_of(x, div=64):
    _, _, h, w = x.shape
    new_h = int(np.ceil(h/div)*div)
    new_w = int(np.ceil(w/div)*div)
    return F.pad(x, (0, new_w-w, 0, new_h-h)), (h, w)
"""
open("/content/utils.py","w").write(utils_py)
print(" Created utils.py")

app_py = r"""
import streamlit as st
import torch
from PIL import Image
import io
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from utils import load_image_as_tensor, tensor_to_pil, compute_metrics, pad_to_multiple_of
from compressai.zoo import bmshj2018_factorized, mbt2018_mean, cheng2020_anchor

# ---------------------------------------
#  Streamlit Config
# ---------------------------------------
st.set_page_config(page_title="Transformer Image Compression", layout="wide")
st.title(" TIC Based Image Compression")
st.caption("Compare multiple pretrained models from CompressAI")

# ---------------------------------------
#  Image Upload
# ---------------------------------------
uploaded = st.file_uploader("Upload an image", type=["png", "jpg", "jpeg"])
if not uploaded:
    st.info("Upload an image to begin.")
    st.stop()

# Get original file size
original_size = len(uploaded.getvalue())
pil_img = Image.open(uploaded).convert("RGB")

#  Limit image size to prevent OOM crash
max_size = 1024
if pil_img.width > max_size or pil_img.height > max_size:
    pil_img.thumbnail((max_size, max_size))
    st.warning(f"Image resized to {pil_img.size} to prevent out-of-memory issues.")

col1, col2, col3 = st.columns(3)
with col1:
    st.image(pil_img, caption=f"Original Image ({pil_img.width}×{pil_img.height})", use_column_width=True)

with col2:
    st.metric("Original File Size", f"{original_size/1024:.2f} KB")

with col3:
    st.metric("Original Size", f"{original_size:,} bytes")

# ---------------------------------------
#  Model Selector
# ---------------------------------------
st.markdown("###  Model Configuration")
model_name = st.selectbox(
    "Choose a pretrained model",
    [
        "bmshj2018_factorized (baseline)",
        "mbt2018_mean (balanced)",
        "cheng2020_anchor (stronger)",
    ],
    help="Select the neural compression model to use"
)

model_map = {
    "bmshj2018_factorized (baseline)": bmshj2018_factorized,
    "mbt2018_mean (balanced)": mbt2018_mean,
    "cheng2020_anchor (stronger)": cheng2020_anchor,
}

# ---------------------------------------
#  Quality Levels
# ---------------------------------------
st.markdown("###  Compression Settings")
qualities = st.multiselect(
    "Select quality levels to compare (1–6):",
    list(range(1, 7)),
    default=[1, 3, 5],
    help="Higher quality = better reconstruction but larger file size"
)

if not qualities:
    st.warning("Please select at least one quality level.")
    st.stop()

# ---------------------------------------
#  Compression Function
# ---------------------------------------
def compress_and_measure(model, x, original_size):
    '''Compress image and measure compressed size'''
    try:
        # Compress the image
        with torch.no_grad():
            out_enc = model.compress(x)

            # Calculate compressed size - handle different string formats
            comp_bytes = 0
            if isinstance(out_enc["strings"], list):
                for string_list in out_enc["strings"]:
                    if isinstance(string_list, list):
                        for s in string_list:
                            comp_bytes += len(s)
                    else:
                        comp_bytes += len(string_list)
            else:
                comp_bytes = len(out_enc["strings"])

            # Decompress to get reconstructed image
            out_dec = model.decompress(out_enc["strings"], out_enc["shape"])
            x_hat = out_dec["x_hat"]

            return comp_bytes, x_hat, True

    except Exception as e:
        st.error(f"Compression error: {e}")
        # Return a fallback size estimate
        fallback_size = max(original_size // 10, 1000)  # 10% of original or 1KB minimum
        return fallback_size, x, False

# ---------------------------------------
#  Run Compression at Each Quality
# ---------------------------------------
device = "cuda" if torch.cuda.is_available() else "cpu"
st.info(f"Using device: {device.upper()}")

model_fn = model_map[model_name]
results = []
compressed_images = {}

progress_bar = st.progress(0)
status_text = st.empty()

for i, q in enumerate(qualities):
    status_text.text(f"Compressing at quality {q}...")

    with st.spinner(f"Loading model for quality {q}..."):
        try:
            model = model_fn(quality=q, pretrained=True).eval().to(device)
        except Exception as e:
            st.error(f"Failed to load model for quality {q}: {e}")
            continue

    x = load_image_as_tensor(pil_img).to(device)
    x_padded, orig_size = pad_to_multiple_of(x, div=64)
    h, w = orig_size

    try:
        # Compress and get size
        comp_bytes, x_hat_padded, success = compress_and_measure(model, x_padded, original_size)

        # Crop back to original size
        x_hat = x_hat_padded.clamp(0, 1)[..., :h, :w]
        rec_img = tensor_to_pil(x_hat)

        # Calculate metrics
        psnr, ssim = compute_metrics(pil_img, rec_img)
        bpp = (comp_bytes * 8) / (pil_img.width * pil_img.height)
        saved_percent = 100 * (1 - (comp_bytes / original_size)) if original_size > 0 else 0
        compression_ratio = original_size / comp_bytes if comp_bytes > 0 else 1

        # Ensure reasonable values
        comp_bytes = max(comp_bytes, 100)  # Minimum 100 bytes
        saved_percent = min(max(saved_percent, -1000), 1000)  # Reasonable bounds
        compression_ratio = min(max(compression_ratio, 0.1), 1000)  # Reasonable bounds

        results.append({
            "Quality": q,
            "Bits-per-pixel": bpp,
            "PSNR (dB)": psnr,
            "SSIM": ssim,
            "Size Saved (%)": saved_percent,
            "Compressed Size (bytes)": comp_bytes,
            "Compressed Size (KB)": comp_bytes / 1024,
            "Compression Ratio": compression_ratio,
            "Success": success
        })

        compressed_images[q] = rec_img

        if success:
            st.success(f" Quality {q}: {comp_bytes/1024:.2f} KB (Saved: {saved_percent:.1f}%)")
        else:
            st.warning(f" Quality {q}: Estimated size {comp_bytes/1024:.2f} KB")

    except Exception as e:
        st.error(f" Compression failed for quality {q}: {str(e)}")
        continue

    progress_bar.progress((i + 1) / len(qualities))

status_text.text("Compression complete!")
progress_bar.empty()

if not results:
    st.error("No successful compressions. Please try different settings.")
    st.stop()

# ---------------------------------------
#  Display Results
# ---------------------------------------
st.markdown("##  Compression Results")

df = pd.DataFrame(results)

# Metrics table
st.markdown("### 📋 Detailed Compression Metrics")
styled_df = df.style.format({
    "Bits-per-pixel": "{:.4f}",
    "PSNR (dB)": "{:.2f}",
    "SSIM": "{:.4f}",
    "Size Saved (%)": "{:.1f}%",
    "Compressed Size (bytes)": "{:,}",
    "Compressed Size (KB)": "{:.2f}",
    "Compression Ratio": "{:.2f}x"
}).background_gradient(subset=['PSNR (dB)', 'SSIM'], cmap='Blues')\
 .background_gradient(subset=['Size Saved (%)'], cmap='Greens')\
 .background_gradient(subset=['Compression Ratio'], cmap='Reds')

st.dataframe(styled_df, use_container_width=True)

# ---------------------------------------
#  Size Comparison Dashboard
# ---------------------------------------
st.markdown("##  Size Comparison Dashboard")

if len(results) > 0:
    best_compression_idx = df['Compressed Size (bytes)'].idxmin()
    best_quality_idx = df['PSNR (dB)'].idxmax()

    col1, col2, col3, col4 = st.columns(4)
    with col1:
        st.metric("Original Size", f"{original_size/1024:.2f} KB")
    with col2:
        min_size = df['Compressed Size (KB)'].min()
        st.metric("Smallest Compressed", f"{min_size:.2f} KB")
    with col3:
        max_ratio = df['Compression Ratio'].max()
        st.metric("Best Compression Ratio", f"{max_ratio:.2f}x")
    with col4:
        avg_saved = df['Size Saved (%)'].mean()
        st.metric("Average Size Saving", f"{avg_saved:.1f}%")

    # ---------------------------------------
    # Size Visualization
    # ---------------------------------------
    st.markdown("###  Size Comparison Charts")

    col1, col2 = st.columns(2)

    with col1:
        # Size comparison bar chart
        fig1, ax1 = plt.subplots(figsize=(10, 6))
        original_kb = original_size / 1024
        compressed_kb = df['Compressed Size (KB)'].values

        x = np.arange(len(qualities))
        width = 0.35

        bars1 = ax1.bar(x - width/2, [original_kb] * len(qualities), width,
                       label=f'Original ({original_kb:.2f} KB)', alpha=0.7, color='blue')
        bars2 = ax1.bar(x + width/2, compressed_kb, width, label='Compressed', alpha=0.7, color='orange')

        ax1.set_xlabel('Quality Level')
        ax1.set_ylabel('File Size (KB)')
        ax1.set_title('Original vs Compressed File Sizes')
        ax1.set_xticks(x)
        ax1.set_xticklabels([f'Q{q}' for q in qualities])
        ax1.legend()
        ax1.grid(True, alpha=0.3)

        # Add value labels on bars
        for bar in bars2:
            height = bar.get_height()
            ax1.text(bar.get_x() + bar.get_width()/2., height + 0.1,
                    f'{height:.1f}KB', ha='center', va='bottom', fontsize=9)

        plt.tight_layout()
        st.pyplot(fig1)

    with col2:
        # Compression ratio chart
        fig2, ax2 = plt.subplots(figsize=(10, 6))
        ratios = df['Compression Ratio'].values

        bars = ax2.bar(x, ratios, color='green', alpha=0.7)
        ax2.set_xlabel('Quality Level')
        ax2.set_ylabel('Compression Ratio')
        ax2.set_title('Compression Ratio by Quality Level')
        ax2.set_xticks(x)
        ax2.set_xticklabels([f'Q{q}' for q in qualities])
        ax2.grid(True, alpha=0.3)

        # Add value labels on bars
        for bar in bars:
            height = bar.get_height()
            ax2.text(bar.get_x() + bar.get_width()/2., height + 0.1,
                    f'{height:.2f}x', ha='center', va='bottom', fontsize=9)

        plt.tight_layout()
        st.pyplot(fig2)

    # ---------------------------------------
    # Visual Comparison
    # ---------------------------------------
    st.markdown("###  Visual Comparison")

    selected_quality = st.selectbox("Select quality to view reconstructed image:", qualities)
    if selected_quality in compressed_images:
        rec_img = compressed_images[selected_quality]
        selected_data = df[df['Quality'] == selected_quality].iloc[0]

        col1, col2 = st.columns(2)
        with col1:
            st.image(pil_img, caption=f"Original Image ({original_size/1024:.2f} KB)", use_column_width=True)
        with col2:
            st.image(rec_img,
                    caption=f"Reconstructed Q{selected_quality} ({selected_data['Compressed Size (KB)']:.2f} KB, "
                           f"Saved: {selected_data['Size Saved (%)']:.1f}%)",
                    use_column_width=True)

        # Show compression info
        col1, col2, col3 = st.columns(3)
        with col1:
            st.metric("Size Reduction", f"{selected_data['Size Saved (%)']:.1f}%")
        with col2:
            st.metric("Compression Ratio", f"{selected_data['Compression Ratio']:.2f}x")
        with col3:
            st.metric("Quality Score", f"{selected_data['PSNR (dB)']:.2f} dB")

        # Show difference map
    st.markdown("####  Difference Map")
    diff_fig, ax = plt.subplots(1, 1, figsize=(4, 4))
    orig_arr = np.array(pil_img).astype(float)
    rec_arr = np.array(rec_img).astype(float)
    diff = np.abs(orig_arr - rec_arr)
    im = ax.imshow(diff.mean(axis=2), cmap='hot')
    ax.set_title(f"Absolute Difference Map (Quality {selected_quality})")
    ax.axis('off')
    plt.colorbar(im, ax=ax, label='Difference Intensity')
    st.pyplot(diff_fig)



    # ---------------------------------------
    #  Download Section
    # ---------------------------------------
    st.markdown("## 💾 Download Results")

    col1, col2 = st.columns(2)

    with col1:
        # Download reconstructed image
        if selected_quality in compressed_images:
            rec_img = compressed_images[selected_quality]
            buf = io.BytesIO()
            rec_img.save(buf, format="PNG", optimize=True)
            st.download_button(
                label=f"Download Reconstructed (Q{selected_quality}) - {selected_data['Compressed Size (KB)']:.1f} KB",
                data=buf.getvalue(),
                file_name=f"reconstructed_q{selected_quality}.png",
                mime="image/png",
            )

    with col2:
        # Download enhanced metrics CSV
        csv_data = df.to_csv(index=False).encode("utf-8")
        st.download_button(
            label="Download Full Metrics (CSV)",
            data=csv_data,
            file_name="compression_metrics.csv",
            mime="text/csv",
        )

# ---------------------------------------
# 📚 Model Information
# ---------------------------------------
st.markdown("## 📚 Model Information")

model_descriptions = {
    "bmshj2018_factorized (baseline)": '''
    **BMSE2018-factorized**: Baseline model using factorized priors for entropy modeling.
    Good balance between complexity and performance.
    ''',
    "mbt2018_mean (balanced)": '''
    **MBT2018-mean**: Uses mean-scale hyperpriors for improved entropy modeling.
    Better performance than factorized priors with moderate complexity.
    ''',
    "cheng2020_anchor (stronger)": '''
    **Cheng2020-anchor**: State-of-the-art model using anchor-based quantization and
    Gaussian mixture models for entropy modeling. Provides excellent rate-distortion performance.
    '''
}

st.info(model_descriptions[model_name])
"""
open("/content/app.py","w").write(app_py)
print("✅ Created fixed app with compression function inside app.py")

 Created utils.py
✅ Created fixed app with compression function inside app.py


In [7]:
from pyngrok import ngrok
import time

ngrok.kill()
public_url = ngrok.connect(8501)
print(f"🌍 Streamlit App is live at: {public_url.public_url}")

# Start Streamlit server
!streamlit run /content/app.py --server.port 8501 > /content/logs.txt 2>&1 &

# Wait for it to start
time.sleep(5)
!tail -n 15 /content/logs.txt

🌍 Streamlit App is live at: https://mirkily-acarpous-dion.ngrok-free.dev



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.45.77.210:8501

